## 範例重點
* 學會從 Keras 內建函式取得資料
* 能夠建立簡易的深度學習模型
* 觀察並了解 Overfit 現象

In [1]:
import os
from tensorflow import keras

# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# 從 Keras 的內建功能中，取得 train 與 test 資料集
# 在本例中，我們將train作為訓練資料、test作為測試資料
train, test = keras.datasets.cifar10.load_data()

In [3]:
# 將 X 與 Y 獨立放進變數
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - 標準化
x_train = x_train / 255.
x_test = x_test / 255.

# 將資料從圖形 (RGB) 轉為向量 (Single Vector)
x_train = x_train.reshape((len(x_train), -1))
x_test = x_test.reshape((len(x_test), -1))

# 將目標轉為 one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [4]:
x_train.shape
x_test.shape

(10000, 3072)

In [5]:
# 建立一個有三層隱藏層之神經網路
def build_mlp():
    input_layer = keras.layers.Input([x_train.shape[-1]])
    x = keras.layers.Dense(units=512, activation="relu")(input_layer)
    x = keras.layers.Dense(units=256, activation="relu")(x)
    x = keras.layers.Dense(units=128, activation="relu")(x)
    out = keras.layers.Dense(units=10, activation="softmax")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
model = build_mlp()
# 用 Keras 內建方法檢視模型各層參數量
model.summary()

optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
dense (Dense)                (None, 512)               1573376   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 訓練模型並檢視驗證集的結果
model.fit(x_train[:30000], y_train[:30000], 
          epochs=100, 
          batch_size=256, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Epoch 1/100
118/118 [==============================] - 4s 35ms/step - loss: 2.0353 - accuracy: 0.2634 - val_loss: 1.8424 - val_accuracy: 0.3438
Epoch 2/100
118/118 [==============================] - 1s 6ms/step - loss: 1.7905 - accuracy: 0.3590 - val_loss: 1.7574 - val_accuracy: 0.3772
Epoch 3/100
118/118 [==============================] - 1s 6ms/step - loss: 1.7083 - accuracy: 0.3886 - val_loss: 1.7595 - val_accuracy: 0.3882
Epoch 4/100
118/118 [==============================] - 1s 6ms/step - loss: 1.6431 - accuracy: 0.4110 - val_loss: 1.6231 - val_accuracy: 0.4197
Epoch 5/100
118/118 [==============================] - 1s 6ms/step - loss: 1.5961 - accuracy: 0.4300 - val_loss: 1.5983 - val_accuracy: 0.4288
Epoch 6/100
118/118 [==============================] - 1s 6ms/step - loss: 1.5396 - accuracy: 0.4484 - val_loss: 1.6349 - val_accuracy: 0.4190
Epoch 7/100
118/118 [==============================] - 1s 6ms/step - loss: 1.5127 - accuracy: 0.4589 - val_loss: 1.5465 - val_accuracy: 0.451

In [ ]:
# 以視覺畫方式檢視訓練過程
import matplotlib.pyplot as plt
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

## 作業
1. 請將 Epoch 加到 500 個，並觀察 learning curve 的走勢
2. 請將 Optimizer 換成 SGD，並觀察 learning curve 的走勢